In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from data_generation import Three_body_2D_Rick
from data_generation import tbp_util
from data_generation.tbp_energy_calculations import visualize_dataset

# Experiment 2

Same as experiment 1 but with 1 change:
- predict delta's

sweep learning rates?
Up layers and neurons?
Go back to original question:
- can the network predict t=10 just as well as t0.1=100 just as well ass t10e-5=10e6

In [ ]:
experiments = {
    "Experiment_2": {
        "max_datasets": 102,
        "prediction_offset": 10,  # predict t time steps ahead
        "batch_size": 256,
        "epochs": 20,
        "validation_split": 0.1,
        "no_dense_layers": 10,
        "neurons_per_layer": 128,
        "learning_rate": 0.0001,
        "dataset": {
            "name": "breen-et-al-00001",
            # filter 1 in 10 values to reduce RAM usage
            "downsample_factor": 10,
            "dataset_index": 1,
            "delta_scaling_factor": 10000
        }
    },
    "Experiment_2_1": {
        "max_datasets": 102,
        "prediction_offset": 10,  # predict t time steps ahead
        "batch_size": 256,
        "epochs": 20,
        "validation_split": 0.1,
        "no_dense_layers": 10,
        "neurons_per_layer": 128,
        "learning_rate": 0.00001,
        "dataset": {
            "name": "breen-et-al-00001",
            # filter 1 in 10 values to reduce RAM usage
            "downsample_factor": 10,
            "dataset_index": 2,
            "delta_scaling_factor": 10000
        }
    }
}

In [ ]:
experiment_id = "Experiment_2_1"
config_name = experiments[experiment_id]["dataset"]["name"]
tbp_util.use_config(config_name)

In [ ]:
# allow autoreloading of imported modules whenever running a cell
# if not included, a kernel restart is needed whenever one of the imports is modified
# (Needs experimenting / import statements to work)
# %load_ext autoreload
# %autoreload 2

In [ ]:
# todo read from config file
# also see tbp_util.py
G = 1.0
M = np.array([1.0, 1.0, 1.0])

Choose example trajectory from the data set to visualize.

In [ ]:
example_dataset = "325"


In [ ]:
visualize_dataset(*tbp_util.load_dataset(example_dataset), G, M)

In [ ]:
x, y, vx, vy = tbp_util.load_dataset(example_dataset)
x[0, :]

In [ ]:
y[0, :]

# deltas

In [ ]:
def get_deltas(x, y, vx, vy, delta=1, scaling_factor=1):
    dx = (x[:-delta] - x[delta:]) * scaling_factor
    dy = (y[:-delta] - y[delta:]) * scaling_factor
    dvx = (vx[:-delta] - vx[delta:]) * scaling_factor
    dvy = (vy[:-delta] - vy[delta:]) * scaling_factor
    return dx, dy, dvx, dvy


In [ ]:
deltas = get_deltas(x, y, vx, vy, 1, 100000)
plt.figure()
plt.boxplot(deltas[0], showfliers=False)
plt.legend(("x1 delta", "x2 delta", "x3 delta"))
plt.show()
plt.figure()
plt.boxplot(deltas[2], showfliers=False)
plt.legend(("vx1 delta", "vx2 delta", "vx3 delta"))
plt.show()

In [ ]:
from tensorflow import keras

keras.backend.set_floatx('float64')
keras.backend.floatx()

# Load training data

Load training data.
A trajectory has t timesteps. For each time step it has 12 variables:
- x, y, vx, vy for each of the 3 bodies

max_datasets: maximum number of datasets to load and train on
prediction offset: how many time steps to predict into the future
downsample factor: only use one in downsample_factor data points for training, allows for k-partition validation
dataset_index: if using a downsample factor of 10, there are 10 unique sub-datasets that can be used

In [ ]:
max_datasets = experiments[experiment_id]['max_datasets']
prediction_offset = experiments[experiment_id]['prediction_offset']
downsample_factor = experiments[experiment_id]['dataset']['downsample_factor']
dataset_index = experiments[experiment_id]['dataset']['dataset_index']
scaling_factor = experiments[experiment_id]['dataset']['delta_scaling_factor']

x_train = np.ndarray((0, 12), dtype=np.float64)
y_train = np.ndarray((0, 12), dtype=np.float64)
for dataset, x, y, vx, vy in tbp_util.load_datasets(limit=max_datasets):
    x = x[dataset_index:, :]
    y = y[dataset_index:, :]
    vx = vx[dataset_index:, :]
    vy = vy[dataset_index:, :]

    input_data = np.column_stack((x, y, vx, vy))
    input_data = input_data[:-prediction_offset:downsample_factor, :]

    deltas = get_deltas(x, y, vx, vy, delta=prediction_offset, scaling_factor=scaling_factor)
    output_data = np.column_stack(deltas)[::downsample_factor, :]

    x_train = np.concatenate((x_train, input_data))
    y_train = np.concatenate((y_train, output_data))


In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
assert x_train.shape == y_train.shape

first training example **[ x1, x2, x3, y1, y2, y3, vx1, vx2, vx3, vy1, vy2, vy3 ]**
first testing  example **[ x1', x2', x3', y1', y2', y3', vx1', vx2', vx3', vy1', vy2', vy3' ]**

In [ ]:
x_train[0,]

In [ ]:
y_train[0,]

# Initialize model

The model consists of 10 layers with 128 neurons. Initial testing has shown this model to be able to over-fit.
The layers use a ReLU activation function and the 12 output neurons a linear function.

The model has be trained on batches of size 256. 10% of the data set is set aside for validation.

This set-up was used to do a preliminary search for network architecture:
- learning rate
- n layers, neurons, batch size, loss function

# Fit model
After every epoch, the model is automatically backed-up, so you can interrupt the kernel and go to validation steps during training if you're curious! Then just re-run the model.fit() step to continue training (might need some testing, lost the loss curve before)

In [ ]:
batch_size = experiments[experiment_id]['batch_size']
epochs = experiments[experiment_id]['epochs']
validation_split = experiments[experiment_id]['validation_split']
learning_rate = experiments[experiment_id]['learning_rate']
steps_per_epoch = round((x_train.shape[0] * (1 - validation_split)) / batch_size)

print(f"{batch_size} {epochs} {validation_split} {learning_rate}")

In [ ]:
def create_model() -> keras.models.Sequential:
    neurons = experiments[experiment_id]['neurons_per_layer']
    no_dense_layers = experiments[experiment_id]['no_dense_layers']
    # start with input layer
    layers = [keras.layers.Dense(neurons, activation=keras.activations.relu, input_shape=[12])]
    # add dense layers
    layers.extend([keras.layers.Dense(neurons, activation=keras.activations.relu) for _ in range(no_dense_layers - 1)])
    # add output layer
    layers.append(keras.layers.Dense(12, activation=keras.activations.linear)
                  )

    return keras.Sequential(layers)

In [ ]:
model = create_model()
model.compile(
    keras.optimizers.Adam(learning_rate=learning_rate),
    loss='mae',
    metrics=['mae', 'mse']
)

In [ ]:
hist_callback = keras.callbacks.History()
callbacks = [
    hist_callback,
    keras.callbacks.BackupAndRestore(backup_dir="model_backup")
]

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks
)

# Save model for later

And plot of the loss as the epochs progress.

In [ ]:
import os

model_id = f'{experiment_id}-{config_name}'
model_path = f'./models/{experiment_id}/{config_name}'
os.makedirs(model_path, exist_ok=True)

In [ ]:
model.save(f"{model_path}/{model_id}_model")

for x in ["h5"]:
    model.save(f"{model_path}/{model_id}_model.{x}", save_format=x)

for x in ["h5", "tf"]:
    model.save_weights(f"{model_path}/{model_id}_weights.{x}", save_format=x)

In [ ]:
os.makedirs(f'{model_path}/{model_id}')
plt.figure()
plt.title(f"Loss graph for {experiment_id}")
plt.plot(hist_callback.history['loss'], label="loss")
plt.savefig(f'{model_path}/{model_id}/loss.svg', format='svg', dpi=1200)
plt.show()

In [ ]:
model_path

In [ ]:
import json
with open(f'{model_path}/{model_id}/loss.json', 'w') as f:
    f.write(json.dumps(hist_callback.history))

# Quick validation set-up

Choose a trajectory from the dataset.
Visualize trajectory
use model to predict the same trajectory ( but in fewer steps! )
compare the 2 trajectory plots
# todo compare the loss or something
# todo save results of validation

In [ ]:
dataset_to_predict = '72'
dataset_to_predict = '18'
x, y, vx, vy = tbp_util.load_dataset(dataset_to_predict)
print(x.shape)

In [ ]:
length_to_predict = int(x.shape[0] / prediction_offset)
print(
    f"The original trajectory is T={x.shape[0]} time steps long, so we have to predict T/prediction_offset={x.shape[0]}/{prediction_offset}={length_to_predict} steps because we predict {prediction_offset} steps into the future.")

In [ ]:
limit = length_to_predict
y_pred = np.zeros((limit, 12), dtype=np.float64)
y_pred[0,] = np.concatenate((x[0,], y[0,], vx[0,], vy[0,]))

In [ ]:
for i in range(limit - 1):
    prediction = model(y_pred[i,].reshape(1, 12), training=False).numpy()

    # convert the delta's to an actual prediction
    prediction /= scaling_factor
    prediction = y_pred[i,].reshape(1, 12) - prediction

    # stop early when the system gets out of bounds
    if np.min(prediction[0, :6]) < -3 or np.max(prediction[0, :6]) > 3 or np.min(prediction) < -20 or np.max(
            prediction) > 20:
        print(f"Stop predicting at t={i * prediction_offset} ({i} steps) after encountering {prediction}")
        break

    y_pred[i + 1,] = prediction

y_pred2 = y_pred[:i]

In [ ]:
# Real trajectory
visualize_dataset(*tbp_util.load_dataset(dataset_to_predict), G, M, down_sample_factor=prediction_offset)

In [ ]:
# Predicted trajectory
pred_x, pred_y, pred_vx, pred_vy = np.hsplit(y_pred, 4)
visualize_dataset(pred_x, pred_y, pred_vx, pred_vy, G, M, down_sample_factor=1)

In [ ]:
import Three_body_2D_Rick

# Comparison plot
true_x, true_y, _, _ = tbp_util.load_dataset(dataset_to_predict)
Three_body_2D_Rick.compare_plot(true_x, true_y, pred_x, pred_y,path=f'{model_path}/{model_id}', savefig=True)